In [ ]:
#####################################################################################
## Project: macaque SP brain                                                       ##
## Script Purpose: generate species peak matrix                                    ##
## Data: 2022.11.02                                                                ##
## Author: Yiming Sun                                                              ##
#####################################################################################

#sleep
ii <- 1
while(1){
  cat(paste("round",ii),sep = "\n")
  ii <- ii+1
  Sys.sleep(30)
}

In [ ]:
#general setting
setwd('/content/data/sunym/project/Brain')
.libPaths('/home/sunym/env/R_4.2.1/lib/R/library')
Sys.setenv(HDF5_USE_FILE_LOCKING=FALSE,RHDF5_USE_FILE_LOCKING=FALSE)

#library
library(Rmisc)
library(Seurat)
library(ggplot2)
library(dplyr)
library(scibet)
library(Matrix)
library(tidyverse)
library(cowplot)
library(viridis)
library(ComplexHeatmap)
library(parallel)
library(ggsignif)
library(RColorBrewer)
library(ggsci)
library(scales)
library(patchwork)
library(ggpointdensity)
library(latex2exp)
library(ArchR)
library(scales)
library(circlize)
library(ggpubr)
library(ggtext)
library(BSgenome.Hsapiens.UCSC.hg38)
library(BSgenome.Mmulatta.UCSC.rheMac10)
library(UpSetR)
library(ggbreak)
library(ggvenn)
library(EnrichedHeatmap)
library(ChIPseeker)
library(org.Hs.eg.db)
library(org.Mmu.eg.db)
library(DESeq2)
library(topGO)
library(clusterProfiler)

#my function
source('https://raw.githubusercontent.com/yimingsun12138/source_list/main/sc_multiomics.R')
source('https://raw.githubusercontent.com/yimingsun12138/source_list/main/genomics.R')
source('/content/script/twilio_send_messages.R')

#initialize ArchR
addArchRThreads(threads = 5)

## generate species homology peaks

In [ ]:
#load ArchR project
Greenleaf_ATAC_ArchR <- loadArchRProject(path = './processed_data/221008_summary/Greenleaf_ATAC_ArchR_221019/')
macaque_multiome_ArchR <- loadArchRProject(path = './processed_data/221008_summary/macaque_multiome_ArchR_221011/')
mouse_multiome_ArchR <- loadArchRProject(path = './processed_data/221008_summary/mouse_multiome_ArchR_221009/')

In [ ]:
#convert macaque homology peaks (mismatch 0.1)
par <- 0.1
query_peakset <- getPeakSet(ArchRProj = macaque_multiome_ArchR)
subject_peakset <- getPeakSet(ArchRProj = Greenleaf_ATAC_ArchR)
query_peakset <- my_unique_peakset_liftover(ori_GRanges = query_peakset,
                                            UCSC_liftOver_path = '~/software/UCSC_liftOver/liftOver',
                                            chain_file = './data/reference/UCSC_chain_file_for_liftOver/rheMac10ToHg38.over.chain',
                                            liftOver_mismatch = par,
                                            length_filter = TRUE,
                                            length_mismatch = par,
                                            chr_filter = TRUE,
                                            mapped_chr = unique(subject_peakset@seqnames),
                                            overlap_filter = TRUE,
                                            tmp_path = '~/temp')
macaque_peakset <- query_peakset

In [ ]:
#convert mouse homology peaks (mismatch 0.4)
par <- 0.4
query_peakset <- getPeakSet(ArchRProj = mouse_multiome_ArchR)
subject_peakset <- getPeakSet(ArchRProj = Greenleaf_ATAC_ArchR)
query_peakset <- my_unique_peakset_liftover(ori_GRanges = query_peakset,
                                            UCSC_liftOver_path = '~/software/UCSC_liftOver/liftOver',
                                            chain_file = './data/reference/UCSC_chain_file_for_liftOver/mm10ToHg38.over.chain',
                                            liftOver_mismatch = par,
                                            length_filter = TRUE,
                                            length_mismatch = par,
                                            chr_filter = TRUE,
                                            mapped_chr = unique(subject_peakset@seqnames),
                                            overlap_filter = TRUE,
                                            tmp_path = '~/temp')
mouse_peakset <- query_peakset

In [ ]:
#merge with human peakset
Brain_peakset <- my_bedtools_merge(peakset_x = macaque_peakset$mapped,
                                   peakset_y =  getPeakSet(ArchRProj = Greenleaf_ATAC_ArchR),
                                   bedtools_path = '~/software/bedtools/bedtools',
                                   tmp_path = '~/temp')
Brain_peakset <- my_bedtools_merge(peakset_x = mouse_peakset$mapped,
                                   peakset_y = Brain_peakset,
                                   bedtools_path = '~/software/bedtools/bedtools',
                                   tmp_path = '~/temp')

In [ ]:
#re-liftover to macaque
par = 0.1
macaque_peakset <- my_unique_peakset_liftover(ori_GRanges = Brain_peakset,
                                              UCSC_liftOver_path = '~/software/UCSC_liftOver/liftOver',
                                              chain_file = './data/reference/UCSC_chain_file_for_liftOver/hg38ToRheMac10.over.chain',
                                              liftOver_mismatch = par,
                                              length_filter = TRUE,
                                              length_mismatch = par,
                                              chr_filter = TRUE,
                                              mapped_chr = unique(getPeakSet(ArchRProj = macaque_multiome_ArchR)@seqnames),
                                              overlap_filter = TRUE,
                                              tmp_path = '~/temp')

In [ ]:
#re-liftover to mouse
par = 0.4
mouse_peakset <- my_unique_peakset_liftover(ori_GRanges = macaque_peakset$ori,
                                            UCSC_liftOver_path = '~/software/UCSC_liftOver/liftOver',
                                            chain_file = './data/reference/UCSC_chain_file_for_liftOver/hg38ToMm10.over.chain',
                                            liftOver_mismatch = par,
                                            length_filter = TRUE,
                                            length_mismatch = par,
                                            chr_filter = TRUE,
                                            mapped_chr = unique(getPeakSet(ArchRProj = mouse_multiome_ArchR)@seqnames),
                                            overlap_filter = TRUE,
                                            tmp_path = '~/temp')

In [ ]:
#generate final peakset
human_peakset <- mouse_peakset$ori
names(macaque_peakset$mapped) <- macaque_peakset$mapped$name
names(mouse_peakset$mapped) <- mouse_peakset$mapped$name

macaque_peakset <- macaque_peakset$mapped[c(names(human_peakset))]
mouse_peakset <- mouse_peakset$mapped[c(names(human_peakset))]

In [ ]:
Brain_ATAC_peak <- SimpleList(human = human_peakset,macaque = macaque_peakset,mouse = mouse_peakset)

In [ ]:
#save data
saveRDS(object = Brain_ATAC_peak,file = './res/step_73_fig_221102/Brain_ATAC_peak.rds')

## generate human peak matrix

In [ ]:
#load data
Brain_ATAC_peak <- readRDS(file = './res/step_73_fig_221102/Brain_ATAC_peak.rds')
Greenleaf_ATAC_ArchR <- loadArchRProject(path = './processed_data/221008_summary/Greenleaf_ATAC_ArchR_221019/')

In [ ]:
#re-craete an ArchR project
temp <- getArrowFiles(ArchRProj = Greenleaf_ATAC_ArchR)
file.exists(temp)
temp <- ArchRProject(
  ArrowFiles = temp,
  outputDirectory = '~/temp/Greenleaf_ATAC_ArchR',
  copyArrows = TRUE,
  geneAnnotation = getGeneAnnotation(ArchRProj = Greenleaf_ATAC_ArchR),
  genomeAnnotation = getGenomeAnnotation(ArchRProj = Greenleaf_ATAC_ArchR)
)

In [ ]:
#count peak matrix
temp <- temp[rownames(Greenleaf_ATAC_ArchR@cellColData)]
temp$cell_type <- Greenleaf_ATAC_ArchR$cell_type
table(temp$cell_type)
temp <- addPeakSet(ArchRProj = temp,peakSet = Brain_ATAC_peak$human)
temp <- addPeakMatrix(ArchRProj = temp)
getAvailableMatrices(ArchRProj = temp)

In [ ]:
#get peak matrix
peak_matrix <- getMatrixFromProject(ArchRProj = temp,useMatrix = 'PeakMatrix',verbose = TRUE)
gene_list <- paste(peak_matrix@rowRanges@seqnames,as.character(peak_matrix@rowRanges@ranges),sep = '-')
peak_matrix <- peak_matrix@assays@data$PeakMatrix
rownames(peak_matrix) <- gene_list

In [ ]:
#save peak_matrix
saveRDS(object = peak_matrix,file = './res/step_73_fig_221102/human_peak_matrix.rds')

In [ ]:
#get grouped peak matrix
temp$sample_cell_type <- paste(temp$Sample,temp$cell_type,sep = '_')
table(temp$sample_cell_type)
grouped_peak_matrix <- getGroupSE(ArchRProj = temp,useMatrix = 'PeakMatrix',
                                  groupBy = 'sample_cell_type',divideN = FALSE)

In [ ]:
#save grouped peak matrix
saveRDS(object = grouped_peak_matrix,file = './res/step_73_fig_221102/human_grouped_peak_matrix.rds')

## generate macaque peak matrix

In [ ]:
#load data
Brain_ATAC_peak <- readRDS(file = './res/step_73_fig_221102/Brain_ATAC_peak.rds')
macaque_multiome_ArchR <- loadArchRProject(path = './processed_data/221008_summary/macaque_multiome_ArchR_221011/')

#re-craete an ArchR project
temp <- getArrowFiles(ArchRProj = macaque_multiome_ArchR)
file.exists(temp)
temp <- ArchRProject(
  ArrowFiles = temp,
  outputDirectory = '~/temp/macaque_multiome_ArchR',
  copyArrows = TRUE,
  geneAnnotation = getGeneAnnotation(ArchRProj = macaque_multiome_ArchR),
  genomeAnnotation = getGenomeAnnotation(ArchRProj = macaque_multiome_ArchR)
)

In [ ]:
#count peak matrix
temp <- temp[rownames(macaque_multiome_ArchR@cellColData)]
temp$cell_type <- macaque_multiome_ArchR$cell_type
table(temp$cell_type)
temp <- addPeakSet(ArchRProj = temp,peakSet = Brain_ATAC_peak$macaque)
temp <- addPeakMatrix(ArchRProj = temp)
getAvailableMatrices(ArchRProj = temp)

In [ ]:
#get peak matrix
peak_matrix <- getMatrixFromProject(ArchRProj = temp,useMatrix = 'PeakMatrix',verbose = TRUE)
gene_list <- paste(peak_matrix@rowRanges@seqnames,as.character(peak_matrix@rowRanges@ranges),sep = '-')
peak_matrix <- peak_matrix@assays@data$PeakMatrix
rownames(peak_matrix) <- gene_list

In [ ]:
#save peak_matrix
saveRDS(object = peak_matrix,file = './res/step_73_fig_221102/macaque_peak_matrix.rds')

In [ ]:
#get grouped peak matrix
temp$sample_cell_type <- paste(temp$Sample,temp$cell_type,sep = '_')
table(temp$sample_cell_type)
grouped_peak_matrix <- getGroupSE(ArchRProj = temp,useMatrix = 'PeakMatrix',
                                  groupBy = 'sample_cell_type',divideN = FALSE)

In [ ]:
#save grouped peak matrix
saveRDS(object = grouped_peak_matrix,file = './res/step_73_fig_221102/macaque_grouped_peak_matrix.rds')

## generate mouse peak matrix

In [ ]:
#load data
Brain_ATAC_peak <- readRDS(file = './res/step_73_fig_221102/Brain_ATAC_peak.rds')
mouse_multiome_ArchR <- loadArchRProject(path = './processed_data/221008_summary/mouse_multiome_ArchR_221009/')

#re-craete an ArchR project
temp <- getArrowFiles(ArchRProj = mouse_multiome_ArchR)
file.exists(temp)
temp <- ArchRProject(
  ArrowFiles = temp,
  outputDirectory = '~/temp/mouse_multiome_ArchR',
  copyArrows = TRUE,
  geneAnnotation = getGeneAnnotation(ArchRProj = mouse_multiome_ArchR),
  genomeAnnotation = getGenomeAnnotation(ArchRProj = mouse_multiome_ArchR)
)

In [ ]:
#count peak matrix
temp <- temp[rownames(mouse_multiome_ArchR@cellColData)]
temp$cell_type <- mouse_multiome_ArchR$Gex_macaque_cell_type
table(temp$cell_type)
temp <- addPeakSet(ArchRProj = temp,peakSet = Brain_ATAC_peak$mouse)
temp <- addPeakMatrix(ArchRProj = temp)
getAvailableMatrices(ArchRProj = temp)

In [ ]:
#get peak matrix
peak_matrix <- getMatrixFromProject(ArchRProj = temp,useMatrix = 'PeakMatrix',verbose = TRUE)
gene_list <- paste(peak_matrix@rowRanges@seqnames,as.character(peak_matrix@rowRanges@ranges),sep = '-')
peak_matrix <- peak_matrix@assays@data$PeakMatrix
rownames(peak_matrix) <- gene_list

In [ ]:
#save peak_matrix
saveRDS(object = peak_matrix,file = './res/step_73_fig_221102/mouse_peak_matrix.rds')

In [ ]:
#get grouped peak matrix
temp$sample_cell_type <- paste(temp$Sample,temp$cell_type,sep = '_')
table(temp$sample_cell_type)
grouped_peak_matrix <- getGroupSE(ArchRProj = temp,useMatrix = 'PeakMatrix',
                                  groupBy = 'sample_cell_type',divideN = FALSE)

In [16]:
#save grouped peak matrix
saveRDS(object = grouped_peak_matrix,file = './res/step_73_fig_221102/mouse_grouped_peak_matrix.rds')